In [ ]:
#load required modules
%matplotlib inline
import sys
import matplotlib as mpl
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import pandas as pd
import datetime
from sodapy import Socrata
import requests
import numpy as np
import re
import seaborn as sns
import  simplejson as json
import scipy
import sklearn
from datetime import timedelta, date, time, datetime as dt
import datetime
import shapely
import shapely.geometry as geom
import holidays
import psycopg2
from io import StringIO
from ediblepickle import checkpoint
import string
import ipywidgets


In [ ]:
#create functions to aplly to pandas 

lat_long=re.compile('(-\d*.?\.\d*.?),\s(\d*.?\.\d*.?)]')
from dateutil import tz
def local2utc(d):
    from_zone=tz.gettz('America/New_York')
    to_zone=tz.gettz('UTC')
    d.replace(tzinfo=from_zone)
    return d.astimezone(to_zone)

def isdem(permnumber):
    #if a permit starts with DEM its a demolition permit
    dem=re.compile('DEM')
    if dem.match(permnumber)==None:
        return False
    else:
        return True
    
def isuse(permnumber):
    #if a permit starts with USE  its a use permit
    use=re.compile('USE')
    if use.match(permnumber)==None:
        return False
    else:
        return True
def make_lower(nh):
    #make neighborhood names lowercase to ensure match
    if type(nh)==str:
        return nh.lower()
    else:
        return nh
    
def is_drug_charge(row):
    #determine if a drug charge has cds in description or is coded 87 (narcotics)
    cds=re.compile('cds',re.I)
    try:
        if row['incidento'][0:1]=='87':
            return True
        elif cds.search(row['chargedescription'])!=None:
            return True
        else:
            return False
    except:
        return False

def pull_year(dt_obj):
    #determine year from date for group by function
    return dt_obj.year

def pull_date(dt_obj):
    #determine year from date for group by function
    return dt_obj.date

def r2(fit, x, y):
    #calculate R2 from a fit and the input data
    p=np.poly1d(fit)
    yhat=p(x)
    ybar=np.sum(y)/len(y)
    ssres=np.sum((yhat-y)**2)
    sstotal=np.sum((y-ybar)**2)
    rsquared=1-ssres/sstotal
    return rsquared

def str2dt(s):
    return dt.strptime(s,'%Y-%m-%dT%H:%M:%SZ')

def str2time(s):
    return dt.strptime(s,'%H:%M:%S %p').time()

def label_coord(coord):
    #lookup grid no from coordinates, can be applied to pandas
    if coord in coord_labels.keys():
        return coord_labels[coord]
    else:
        return np.nan

def label2coord(lab):
    #lookup coordinates from grid no
    if np.isfinite(lab):
        return tuple(zip(labels_coord[lab]))
    else:
        return tuple(zip(tuple((np.nan,np.nan))))   
    
def daterange(start_date, end_date):
    #create a linear range of dates
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def hourrange(start_date, end_date):
    for n in range(int ((end_date-start_date).total_seconds()/60**2)):
        yield start_date+timedelta(hours=n)

def nan2zero(o):
    #convert nan to zero, typicaly applies if nan is created from a merge between records
    #and the gid attributes dataframe if none of the record contain the grid point
    if np.isnan(o):
        return  0
    else:
        return o
    
def pull_lat(location):
    #pull latitude from location given in dict format, check if saved in csv file as string
    if type(location)==str:
        location=lat_long.search(location)
    try:
        return float(location.groups(0)[0])
    except Exception:
        return np.nan

def pull_lon(location):
    #pull longitude from location given in dict format, check if saved in csv file as string
    if type(location)==str:
        location=lat_long.search(location)
    try:
        return float(location.groups(0)[1])
    except Exception:
        return np.nan

def gc_distance(lat1, lon1, lat2, lon2):
    #calculate distance between two coordinateds using the haversine formula
    lat1=lat1*2*np.pi/360
    lat2=lat2*2*np.pi/360
    lon1=lon1*2*np.pi/360
    lon2=lon2*2*np.pi/360
    dphi=lat1-lat2
    dlambda=lon1-lon2
    arc=2*np.arcsin((np.sin(dphi/2)**2+np.cos(lat1)*np.cos(lat2)*np.sin(dlambda/2)**2)**.5)
    return 3959*arc

def place_on_grid(df, location_key):
    #convert latitude and longitude given in a location dict into the grid
    df['latitude']=df[location_key].apply(pull_lat)
    df['longitude']=df[location_key].apply(pull_lon)
    df['lat_bin']=pd.cut(df['latitude'],bins=lats,labels=lat_label)
    df['lon_bin']=pd.cut(df['longitude'],bins=lons,labels=lon_label)
    df['grid_no']=df.apply(lambda x: label_coord(tuple((x['lat_bin'], x['lon_bin']))), axis=1)
    df['grid_coords']=df['grid_no'].apply(label2coord)
    return df

def obj_min_dist(a,b):
    return(a.distance(b)/1609.2446469)

# Load Data From APIs

## Dataset Data from open Baltimore

### Define functions to pickle the data from the API's

In [ ]:
from ediblepickle import checkpoint
import string

def r2p_pickler(df,f):
    df.to_csv(f,encoding='UTF-8')
    
def r2p_unpickler(f):
    return pd.DataFrame.from_csv(f)

@checkpoint(key=string.Template('/home/vagrant/Baltimore_vacants_crime/{0}.csv'), 
            pickler=r2p_pickler, unpickler=r2p_unpickler, refresh=False)
def records_to_pandas(name,ds,lim):
    results = client.get(ds,limit=lim)
    
    return pd.DataFrame.from_records(results)



In [ ]:
secrets = json.loads(open('apikey.json.nogit','r').read())
client = Socrata("data.baltimorecity.gov", secrets['app_token'])

### Victim Based Crime Data

In [ ]:
#download the victim based crime dataset make into dataframe, make 
#crimedate into a datetime object, make neighborhood lowercase
vict_crime=records_to_pandas('vict_crime',"4ih5-d5d5",500000)
vict_crime['crimedate']=pd.to_datetime(vict_crime['crimedate'],yearfirst=True)
vict_crime['neighborhood']=vict_crime['neighborhood'].apply(make_lower)


### Arrests Data

In [ ]:
#Download arrest data, determine if it is a drug charge, convert date to 
#datetime object, make neighborhood lower case
arrest_df=records_to_pandas('arrests','icjs-e3jg',160000)
arrest_df.is_drugs=arrest_df.apply(is_drug_charge, axis=1)
arrest_df.arrestdate=pd.to_datetime(arrest_df['arrestdate'],yearfirst=True)
arrest_df['neighborhood']=arrest_df['name1'].apply(make_lower)

### Housing Permit Data

In [ ]:
house_perm_df=pd.DataFrame.from_csv('housing_permits.csv')

In [ ]:
#download housing permits, determine if the are use or demolition permits,
#make permit date into a datetime, make the neighborhood lowercase
perm_res=client.get('9t78-k3wf',limit=600000)
house_perm_df=records_to_pandas('housing_permits','9t78-k3wf',600000)
house_perm_df['dateissue']=pd.to_datetime(house_perm_df.dateissue,yearfirst=True)
house_perm_df['isdem']=house_perm_df.casenum.apply(isdem)
house_perm_df['isuse']=house_perm_df.casenum.apply(isuse)
house_perm_df['neighborhood']=house_perm_df.neighborhood.apply(make_lower)
house_perm_df['latitude']=house_perm_df['location'].apply(pull_lat)
house_perm_df['longitude']=house_perm_df['location'].apply(pull_lon)

### Vacant Buildings Data

In [ ]:
#Dowload vacant house dataset, make neighborhood lowercase
vac_df=records_to_pandas('vacants','rw5h-nvv4',20000)
vac_df.neighborhood=vac_df.neighborhood.apply(make_lower)
vac_df['latitude']=vac_df['location'].apply(pull_lat)
vac_df['longitude']=vac_df['location'].apply(pull_lon)

### LIquor License Data

In [ ]:
liquor_locs=records_to_pandas('liquor','g2jf-x8pp',10000)
liquor_locs['latitude']=liquor_locs['location_1'].apply(pull_lat)
liquor_locs['longitude']=liquor_locs['location_1'].apply(pull_lon)

## Baltimore Open Data (GIS)

### GIS Data Pickler

In [ ]:
def json_pickler(out_dict,f):
    json.dumps(out_dict,f,encoding='UTF-8')
    
def json_unpickler(f):
    return json.loads(open(f).read())

@checkpoint(key=string.Template('/home/vagrant/Baltimore_vacants_crime/{0}.json'),pickler=json_pickler,
           unpickler=json_unpickler, refresh='False')
def get_geojson(name,ds):
    loc='http://gis-baltimore.opendata.arcgis.com/datasets/'+ds+'.geojson'
    return requests.get(loc,secrets['app_token']).json()['features']

## download GIS Data

In [ ]:
## download location of parks and zoning data
parks=get_geojson('parks','583030920e7f41089e01aff63bd305e6_0')
ez=get_geojson('ez','90712f770cac495c8c3e1659e5bf4d71_0')
ezta=get_geojson('ezta','506abeca72cb4365bef0a1f3bbdad7b7_0')
zoning=get_geojson('zoning','5645866640f6436f8f37de3d3fbcbbb3_0')
real_property=get_geojson('real_property','b41551f53345445fa05b554cd77b3732_0')
house_typology=get_geojson('house_typ','7333122beb68414e9e7744f2dd986578_0')
leg_districts=get_geojson('leg_dist','1019cf2276f24c1f98de5696d68f449d_0')
trees=client.get('xfdv-p3cc',limit=100000)

In [ ]:
ezta[1]

### Convert GIS Data into dataframe with max and min lat and long, features (if applicable) and polygon

#### Vegitated Area (no Features, Yes or NO)

In [ ]:
tree_poly=[]
for feat in trees:
    coords=feat['the_geom']['coordinates']
    for shape in coords:
        tuple_list=[tuple(pt) for pt in shape[0]]
        out_dict={}
        out_dict['poly']=shapely.geometry.Polygon(tuple_list)
        out_dict['max_lon']=max(tuple_list)[0]
        out_dict['min_lon']=min(tuple_list)[0]
        out_dict['max_lat']=max(tuple_list,key=lambda (x,y):(y,x))[1]
        out_dict['min_lat']=min(tuple_list,key=lambda (x,y):(y,x))[1]
        tree_poly.append(out_dict)
tree_df=pd.DataFrame(tree_poly)

#### Functions if the GEO_JSON should create a polygon or a point

In [ ]:
def make_poly_and_bounds(shape,prop_dict):
    tuple_list=[tuple(pt) for pt in shape]
    out_dict={}
    out_dict['properties']=prop_dict
    out_dict['poly']=shapely.geometry.Polygon(tuple_list)
    out_dict['max_lon']=max(tuple_list)[0]
    out_dict['min_lon']=min(tuple_list)[0]
    out_dict['max_lat']=max(tuple_list,key=lambda (x,y):(y,x))[1]
    out_dict['min_lat']=min(tuple_list,key=lambda (x,y):(y,x))[1]
    return out_dict

@checkpoint(string.Template('/home/vagrant/Baltimore_vacants_crime/{1}df_from_json.pkd'),refresh=False)
def df_w_polygons_from_geojson(json_dict,key,properties=[]):
    poly=[]
    for feat in json_dict:
        prop_dict={}
        if len(properties)>0:
            for p in properties:
                prop_dict[p]=feat['properties'][p]
        if feat['geometry']['type']=='Polygon':
            poly.append(make_poly_and_bounds(feat['geometry']['coordinates'][0]
                                             ,prop_dict))
        else:
            for shape in feat['geometry']['coordinates']:
                poly.append(make_poly_and_bounds(shape[0],
                                                 prop_dict))
    return pd.DataFrame(poly)

@checkpoint(string.Template('/home/vagrant/Baltimore_vacants_crime/{1}df_from_json.pkd'))
def df_w_rep_points_from_geojson(json_dict,key,properties=[]):
    point=[]
    for feat in json_dict:
        prop_dict={}
        if len(properties)>0:
            for p in properties:
                prop_dict[p]=feat['properties'][p]
        if feat['geometry']['type']=='Polygon':
            coords=feat['geometry']['coordinates'][0]
        else:
            coords=feat['geometry']['coordinates'][0][0]
        tuple_list=[tuple(pt) for pt in coords]
        out_dict={}
        poly=shapely.geometry.Polygon(tuple_list)
        pt=poly.representative_point().coords[0]
        prop_dict['latitude']=pt[0]
        prop_dict['longitude']=pt[1]
        point.append(prop_dict)
    return pd.DataFrame(point)

In [ ]:
leg_dist_df=df_w_polygons_from_geojson(leg_districts,'leg_dist')
park_df=df_w_polygons_from_geojson(parks,['ACRES'])
ez_df=df_w_polygons_from_geojson(ez,'ez')
ezta_df=df_w_polygons_from_geojson(ezta,'ezta')
zoning_df=df_w_polygons_from_geojson(zoning,'zoning',['Zoning'])
rp_df=df_w_rep_points_from_geojson(real_property,'rp',
                                   ['TAXBASE','USEGROUP','DWELUNIT',
                                    'FULLCASH','ZONECODE','OWNER_1'])
house_typ_df=df_w_polygons_from_geojson(house_typology,'ht',['CSP1214_CI','VSP1214_CI','HuSQmi14CI',
                                                        'PCIn_14_CI','CFC1214_CI','PHOO_Hu_14',
                                                        'HMT','CVAC_14_CI','CVaLT14_CI','MSP1214_CI',
                                                        'PCIn_14_CI'
                                                       ])

In [ ]:
len(ezta)

### API for non-baltimore Data (Sunset and Weather)

In [ ]:
sun_df=pd.DataFrame.from_csv('baltimore_sunset.csv')
weather_df=pd.DataFrame.from_csv('bwi_weather.csv')

In [ ]:
start_date = date(2012, 1, 1)
end_date = date.today()
days=(end_date-start_date).days
day_list=[start_date+timedelta(d) for d in xrange(days)]
sun_info=list()
for sd in day_list:
    param={'lat':39.25,'lng':-76.6,'date':sd.strftime('%Y-%m-%d')}
    resp=requests.get('https://api.sunrise-sunset.org/json?',params=param)
    rdict=resp.json()
    sun_dict=rdict['results']
    sun_dict['date']=sd
    sun_info.append(sun_dict)
sun_df=pd.DataFrame(sun_info)
sun_df.to_csv('baltimore_sunset.csv')  

In [ ]:
for years in range(2012,2018):
    param={'stid':'kbwi','start':date(years,1,1).strftime('%Y%m%d%H%M'),
       'end':(date(years,12,31)).strftime('%Y%m%d%H%M'),
      'token':'34d783ecbaa3414e8f942227b22c1ab0'}
    weath_req=requests.get('https://api.mesowest.net/v2/stations/timeseries?',params=param)
    if years==2012:
        wdict=weath_req.json()['STATION'][0]['OBSERVATIONS']
        weather_df=pd.DataFrame(wdict)
        print(weather_df.columns)
    else:
        wdict=weath_req.json()['STATION'][0]['OBSERVATIONS']
        weather_df.append(pd.DataFrame(wdict))
weather_df=weather_df[['date_time','air_temp_set_1','precip_accum_24_hour_set_1','precip_accum_one_hour_set_1','weather_condition_set_1d']]
weather_df.to_csv('bwi_weather.csv')

In [ ]:
## Begin DataFrame PreProcessing

Read Static Data from CSV files to avoid iternet bottleneck

# Look at trends based on the grid and static predictors


## Setup Grid
### Make a 0.1 mile by 0.1 mile grid, with labels at the centroids.

In [ ]:
bmap = Basemap(llcrnrlon=-76.72,llcrnrlat=39.195,urcrnrlon=-76.52,urcrnrlat=39.375,projection='lcc', lat_0=39.25, lon_0=-76.65)
#bmap.readshapefile('/home/kevin/TDI/Neighborhoods1','hoods',drawbounds=True)
lons, lats, x, y = bmap.makegrid(107, 124, returnxy=True)
lats=sorted(lats[:,0])
lons=sorted(lons[0,:])
lon_label=list()
for i,lon in enumerate(lons):
    if i>0:
        lon_label.append((lons[i]+lons[i-1])/2)
lat_label=list()
for i,lat in enumerate(lats):
    if i>0:
        lat_label.append((lats[i]+lats[i-1])/2)

In [ ]:
lat_step=np.mean([lat_label[i+1]-lat_label[i] for i in xrange(122)])
print(lat_step)
lon_step=np.mean([lon_label[i+1]-lon_label[i] for i in xrange(105)])
print(lon_step)

In [ ]:
len(lats)=

### Create Data Frame with Latitude and Longitude point and polygon represneting grid of all locations in the city

In [ ]:
def check_in_city(point):
    pt=geom.Point(point[0],point[1])
    return any([poly.contains(pt) for poly in leg_dist_df['poly']])

def grid_box(point):
    return geom.Polygon([(point[0]-lon_step/2,point[1]+lat_step/2),
                        (point[0]+lon_step/2,point[1]+lat_step/2),
                        (point[0]+lon_step/2,point[1]-lat_step/2),
                        (point[0]-lon_step/2,point[1]-lat_step/2),
                        (point[0]-lon_step/2,point[1]+lat_step/2)])
import itertools
#identify the set of nodes that are in the city
plt.figure(figsize=(24,24))
coord_set=[{'lon':a[0],'lat':a[1],'point':geom.Point(a),'poly':grid_box(a)}
            for a in itertools.product(lon_label,lat_label) if check_in_city(a)]
coord_set_df=pd.DataFrame(coord_set)

### use GeoJSON data to assign information to coordinates data frame for polygons

In [ ]:
def check_contains(poly,point):
    #Check if a point is in a polygon using shapely
    return poly.contains(point)
def find_properties(layer,row_tup):
    #Slice the dataframe of shapely polygons based on bounding box of each polygon.
    #Search all polygons whose bounding box contains the point to determine which polygon 
    #contains the point.  Assign attributes of the polygon to the point.
    row=row_tup[1]
    out_dict={}
    layer_sliced = layer.loc[(layer['max_lat']>row['lat']) & 
                            (layer['min_lat']<row['lat']) & 
                            (layer['max_lon']>row['lon']) &
                            (layer['min_lon']<row['lon']),
                            :].copy()
    if layer_sliced.shape[0]==0:
        return {'I':row_tup[0],'properties':False}
        
    else:
        for feat in layer_sliced.iterrows():
            if feat[1]['poly'].contains(row['point']):
                if 'properties' in feat[1]:
                    out_dict=feat[1]['properties'].copy()
                    out_dict['I']=row_tup[0]
                    return out_dict
                else:
                    return {'I':row_tup[0],'properties':True}
        return {'I':row_tup[0],'properties':False}
                

In [ ]:
@checkpoint(key=string.Template('/home/vagrant/Baltimore_vacants_crime/{1}.pkd'),refresh=True)
def coord_properties_from_polygons(layer,frame_id):
    cp_dict_list=[]
    for row in coord_set_df.iterrows():
        a=(find_properties(layer,row))
        cp_dict_list.append(a)
    return pd.DataFrame(cp_dict_list)

ht_coord_df=coord_properties_from_polygons(house_typ_df,'ht_coord')
park_coord_df=coord_properties_from_polygons(park_df,'park_coord')
zoning_coord_df=coord_properties_from_polygons(zoning_df,'zoning_coord')
ez_coord_df=coord_properties_from_polygons(ez_df,'ez_coord')
ezta_coord_df=coord_properties_from_polygons(ezta_df,'ezta_coord')
tree_coord_df=coord_properties_from_polygons(tree_df,'tree_coord')

In [ ]:
#Rename the columns to more descriptive names
ez_coord_df.rename(columns={'properties':'ent_zone'},inplace=True)
ezta_coord_df.rename(columns={'properties':'targ_ent_zone'},inplace=True)
tree_coord_df.rename(columns={'properties':'vegetated'},inplace=True)
park_coord_df.rename(columns={'properties':'park'},inplace=True)

ht_coord_df.rename(columns={'CSP1214_CI':'residential_sales',
                    'VSP1214_CI':'var_res_sale_price',
                    'MSP1214_CI':'med_res_sale_price',
                    'CVAC_14_CI':'vac_parcels',
                    'PCIn_14_CI':'perc_com_ind',
                    'HMT':'housing_class',
                    'PHOO_Hu_14':'percent_owner_occupied',
                    'CVaLT14_CI':'vac_lots'
                    },inplace=True)


### Combine the attributes from GIS onto a single data frame

In [ ]:
@checkpoint(key='/home/vagrant/Baltimore_vacants_crime/coord_from_gis_poly.pkd',refresh=False)
def join_columns():
    rsuf=['ht','park','zoning','ent_zone','ent_zone_ta','tree']
    layers=[ht_coord_df,park_coord_df,zoning_coord_df,ez_coord_df,ezta_coord_df,tree_coord_df]
    ct=0
    c2=coord_set_df.copy()
    for i,layer in enumerate(layers):
        layer.reindex(columns=['I'])
        c2=c2.join(layer,rsuffix=rsuf[i],how='inner')
    return c2

coord_and_poly=join_columns()
coord_and_poly.drop(['point','poly'],axis=1,inplace=True)
coord_and_poly.shape      
    

## Add Housing Permits Data and Do Preliminary Visualization

### Grab Housing Permit latitude and longitude and put poitns on the grid

In [ ]:
locations_re=re.compile(r'coordinates\'\: \[(-\d.*?), (\d.*?)\]')
def pull_lat(val):
    if isinstance(val,dict):
        return val['coordinates'][1]
    else:
        try:
            return float(locations_re.search(val).groups(0)[1])
        except Exception:
            return np.nan
    
def pull_lon(val):
    if isinstance(val,dict):
        return val['coordinates'][0]
    else:
        try:
            return float(locations_re.search(val).groups(0)[0])
        except Exception:
            return np.nan
        
house_perm_df['latitude']=house_perm_df['location'].apply(pull_lat)
house_perm_df['longitude']=house_perm_df['location'].apply(pull_lon)
house_perm_df['lat_label']=pd.cut(house_perm_df['latitude'],bins=lats,labels=lat_label)
house_perm_df['lon_label']=pd.cut(house_perm_df['longitude'],bins=lons,labels=lon_label)

### Filter housing permit dataset for different aggregations

In [ ]:
def filter_in_list(df, col, vals):
    return df.loc[df[col].isin(vals),:]

def filter_by_re(df,col,regex):
    filter_re=re.compile(regex)
    return df.loc[df[col].apply(lambda x:bool(filter_re.search(x)))]


hp_res=filter_in_list(house_perm_df,'prop_use',['SF','MF','MIXR'])
hp_vac=filter_in_list(house_perm_df,'existing_use',['VAC'])
hp_res_vac=hp_res.join(hp_vac.loc[:,['existing_use']],how='inner',rsuffix='vac')
hp_dem=filter_by_re(house_perm_df,'permitnum','^DEM')
hp_dem_vac=hp_dem.join(hp_vac.loc[:,['existing_use']],how='inner',rsuffix='vac')


### Aggregate the filtered dataframes and add to the main columnm

In [ ]:
def agg_column_and_join(df1,df2,col,lefton,righton, groupers, aggfunc, new_names):
    locs=groupers+col
    df2=df2.loc[:,locs].groupby(groupers).aggregate(aggfunc)
    df2.reset_index(inplace=True)
    df3=df1.merge(df2, left_on=lefton,right_on=righton, how='left',
                 suffixes=('', '_drop'))
    rename=dict()
    for i,c in enumerate(col):
        print(new_names[i])
        rename[c]=new_names[i]
    df3.rename(columns=rename, inplace=True)
    return df3
c3=agg_column_and_join(coord_and_poly,house_perm_df,['cost_est'],['lat','lon'],
                      ['lat_label','lon_label'],['lat_label','lon_label'],np.sum,
                      ['total_permit_cost'])
c3=agg_column_and_join(c3,hp_res,['cost_est'],['lat_label','lon_label'],
                      ['lat_label','lon_label'],['lat_label','lon_label'],np.sum,
                      ['res_permit_cost'])
c3=agg_column_and_join(c3,hp_vac,['cost_est'],['lat_label','lon_label'],
                      ['lat_label','lon_label'],['lat_label','lon_label'],np.sum,
                      ['vac_permit_cost'])
c3=agg_column_and_join(c3,hp_res_vac,['cost_est'],['lat_label','lon_label'],
                      ['lat_label','lon_label'],['lat_label','lon_label'],np.sum,
                      ['vac_res_permit_cost'])
c3=agg_column_and_join(c3,hp_dem,['cost_est'],['lat_label','lon_label'],
                      ['lat_label','lon_label'],['lat_label','lon_label'],np.count_nonzero,
                      ['total_dem'])

### visualizations

In [ ]:
from bokeh.io import  output_notebook, show, push_notebook
from bokeh.models import ColumnDataSource, GMapOptions, ColumnDataSource, Select, ColorBar
from bokeh.models import CDSView, CustomJS, CustomJSFilter, BooleanFilter, ColorMapper
from bokeh.models import LinearColorMapper,LogColorMapper, Title
from bokeh.plotting import gmap, figure
from bokeh import palettes
from bokeh.layouts import row, widgetbox 
from bokeh.resources import INLINE
from bokeh.io import output_notebook, show
from ipywidgets import interact

def update_permit_costs(Permit):
    x=Permit
    col_dict={'All Permits':'total_permit_cost',
               'Residential Permits':'res_permit_cost',
               'Previously Vacant Permits':'vac_permit_cost',
               'Previously Vacant Residential Permits':'vac_res_permit_cost'}
    p.title.text=x
    a.glyph.fill_color['field']=col_dict[x]
    push_notebook(r)
    
output_notebook()
map_options = GMapOptions(lat=39.308, lng=-76.6158, map_type='terrain', zoom=11)
p = gmap(secrets["google_maps_api"], map_options, 
         title="All Building Permit Cost Estimates", width=600, height=400)
mapper = LogColorMapper( palette=palettes.Magma256, low=10**4, high=10**8)
colors= { 'field': 'total_permit_cost', 'transform': mapper}
source=ColumnDataSource(c3)
a=p.square(x='lon',y='lat',color=colors,size=5,source=source,alpha=.4)
color_bar = ColorBar( color_mapper=mapper, location=( 0, 0))
p.add_layout( color_bar, 'right')
interact(update_permit_costs, Permit=['All Permits','Residential Permits','Previously Vacant Permits',
                                 'Previously Vacant Residential Permits'])
r=show(row(p),notebook_handle=True,notebook_url='https://159.65.243.213:8888')




In [642]:
from bokeh.io import  output_notebook, show, push_notebook
from bokeh.models import ColumnDataSource, GMapOptions, ColumnDataSource, Select, ColorBar
from bokeh.models import CDSView, CustomJS, CustomJSFilter, BooleanFilter, ColorMapper
from bokeh.models import LinearColorMapper,LogColorMapper, CategoricalColorMapper
from bokeh.plotting import gmap, figure
from bokeh import palettes
from bokeh.layouts import row, widgetbox 
from bokeh.resources import INLINE
from bokeh.io import output_notebook, show
from ipywidgets import interact

output_notebook()
map_options = GMapOptions(lat=39.308, lng=-76.6158, map_type='terrain', zoom=11)
p2 = gmap(secrets["google_maps_api"], map_options, 
         title="Housing Typology",width=800,height=400)
mapper = CategoricalColorMapper( palette=palettes.Category10_10, factors=sorted(c3['housing_class'].unique()))
colors= { 'field': 'housing_class', 'transform': mapper }
source=ColumnDataSource(c3.loc[:,['lat','lon','housing_class']])
a2=p2.square(x='lon',y='lat',color=colors,size=5,source=source,alpha=.4, legend='housing_class')

ht=show(row(p2),notebook_handle=True,notebook_url='https://159.65.243.213:8888')

Loading BokehJS ...

In [ ]:
c3['var_res_sale_price'].hist()

In [ ]:
c3.columns

In [ ]:
def update_housing_stats(Housing_Stats):
    x=Housing_Stats
    col_dict={'Median Residential Sale Price':['med_res_sale_price',
                                               LogColorMapper( palette=palettes.Magma11, low=10**3, high=10**6)],
               'Variance Residential Sale Price':['var_res_sale_price',
                                              LinearColorMapper( palette=palettes.Magma11, low=0, high=1)]}
    p3.title.text=x
    a3.glyph.fill_color['field'] = col_dict[x][0]
    a3.glyph.line_color['field'] = col_dict[x][0]
    color_bar3.color_mapper=col_dict[x][1]
    push_notebook(r3)
    
map_options = GMapOptions(lat=39.308, lng=-76.6158, map_type='terrain', zoom=11)
p3 = gmap(secrets["google_maps_api"], map_options, 
         title="All Building Permit Cost Estimates", width=600, height=400)
mapper3 = LogColorMapper( palette=palettes.Magma11, low=10**3, high=10**6)
colors= { 'field': 'med_res_sale_price', 'transform': mapper3}
source=ColumnDataSource(c3)
a3=p3.square(x='lon',y='lat',color=colors,size=5,source=source,alpha=.4)
color_bar3 = ColorBar( color_mapper=mapper3, location=( 0, 0))
p3.add_layout( color_bar3, 'right')
interact(update_housing_stats, Housing_Stats=
         ['Median Residential Sale Price','Variance Residential Sale Price'])
                                      
r3=show(row(p3),notebook_handle=True,notebook_url='https://159.65.243.213:8888')




In [ ]:
map_options = GMapOptions(lat=39.308, lng=-76.6158, map_type='terrain', zoom=11)
p4 = gmap(secrets["google_maps_api"], map_options, 
         title="All Building Permit Cost Estimates", width=600, height=400)
mapper4 = LinearColorMapper( palette=palettes.Magma11, low=0, high=1.5)
colors= { 'field': 'var_res_sale_price', 'transform': mapper4}
a4=p4.square(x='lon',y='lat',color=colors,size=5,source=source,alpha=.4)
color_bar4 = ColorBar( color_mapper=mapper4, location=( 0, 0))
p4.add_layout( color_bar4, 'right')
                                      
r4=show(row(p4),notebook_handle=True,notebook_url='https://159.65.243.213:8888')

In [ ]:
map_options = GMapOptions(lat=39.308, lng=-76.6158, map_type='terrain', zoom=11)
p4 = gmap(secrets["google_maps_api"], map_options, 
         title="Percent Owner Occupied", width=600, height=400)
mapper4 = LinearColorMapper( palette=palettes.Magma11, low=0, high=1)
colors= { 'field': 'percent_owner_occupied', 'transform': mapper4}
a4=p4.square(x='lon',y='lat',color=colors,size=5,source=source,alpha=.4)
color_bar4 = ColorBar( color_mapper=mapper4, location=( 0, 0))
p4.add_layout( color_bar4, 'right')
                                      
r4=show(row(p4),notebook_handle=True,notebook_url='https://159.65.243.213:8888')

In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(10,10))
crime_labels=['Murder','Rape','Robbery','Assault','Burglary','Larceny','Auto Theft','Arson','Shooting']
vict_crime['crimecode']=vict_crime['crimecode'].apply(lambda a:a[0])
vict_crime['crimecode'].value_counts(sort=False).sort_index().plot(kind='bar', tick_label=crime_labels)
vict_crime['violent']=vict_crime['crimecode'].apply(lambda a:a in ['1','2','3','4','8','9'])
fig2=plt.figure(figsize=(10,10))
vict_crime['violent'].value_counts(sort=False).sort_index().plot(kind='bar')
mpl.rcParams.update({'font.size': 48})
plt.show()

In [ ]:
vict_crime['latitude']=vict_crime['location_1'].apply(pull_lat)
vict_crime['longitude']=vict_crime['location_1'].apply(pull_lon)
vict_crime['lat_label']=pd.cut(vict_crime['latitude'],bins=lats,labels=lat_label)
vict_crime['lon_label']=pd.cut(vict_crime['longitude'],bins=lons,labels=lon_label)

In [ ]:
def get_year(dt):
    return dt.year
vict_crime['crime_year']=vict_crime['crimedate'].apply(get_year)
vict_crime['inside']=vict_crime['inside_outside'].apply(lambda a:a == 'I')
vict_crime['outside']=vict_crime['inside_outside'].apply(lambda a:a == 'O')

In [ ]:
vict_crime_grouped=vict_crime.groupby(
    ['lat_label','lon_label','crime_year']).aggregate(np.sum)

In [ ]:
vict_crime_grouped=vict_crime_grouped.reset_index()

In [ ]:
vict_crime_grouped.loc[vict_crime_grouped['crime_year']==2012].sum()

In [ ]:
for x in xrange(2012,2018):
    df=vict_crime_grouped.loc[vict_crime_grouped['crime_year']==x,:]
    c3=c3.merge(df, left_on=['lat_label','lon_label'],right_on=['lat_label','lon_label'], how='left',
                 suffixes=('','_'+str(x)))
    

In [ ]:
mp=c3['total_incidents_2017'].hist(

In [650]:
# def update_crime_stats(Crime_Type):
#     x=Crime_Type
#     col_dict={'All Crime 2017':['total_incidents_2017',
#                                                LinearColorMapper( palette=palettes.Magma11, low=0, high=50)],
#                'Violent Crime 2017':['violent_2017',
#                                                LinearColorMapper( palette=palettes.Magma11, low=0, high=30)],
#                'All Crime 2012':['total_incidents_2012',
#                                                LinearColorMapper( palette=palettes.Magma11, low=0, high=50)],
#                'Violent Crime 2012':['violent_2012',
#                                                LinearColorMapper( palette=palettes.Magma11, low=0, high=30)]}
#     p5.title.text=x
#     a5.glyph.fill_color['field'] = col_dict[x][0]
#     a5.glyph.line_color['field'] = col_dict[x][0]
#     color_bar5.color_mapper=col_dict[x][1]
#     push_notebook(r5)
    
    
output_notebook()
map_options = GMapOptions(lat=39.308, lng=-76.6158, map_type='terrain', zoom=12)
p5 = gmap(secrets["google_maps_api"], map_options, 
         title="Violent Crime 2012", width=600, height=500)
mapper5 = LinearColorMapper( palette=palettes.Magma256, low=0, high=30)
colors= { 'field': 'violent_2012', 'transform': mapper5}
source=ColumnDataSource(c3.loc[:,['lat','lon','violent_2012']])
a5=p5.square(x='lon',y='lat',color=colors,size=5,source=source,alpha=.4)
color_bar5 = ColorBar( color_mapper=mapper5, location=( 0, 0))
p5.add_layout( color_bar5, 'right')
#interact(update_crime_stats, Crime_Type=['All Crime 2017','Violent Crime 2017','All Crime 2012','Violent Crime 2012'])
r5=show(row(p5),notebook_handle=True,notebook_url='https://159.65.243.213:8888')

Loading BokehJS ...

In [ ]:
import dill
dill.dump(c3,open('dataframe_bmore.pkd','w'))